# 매치 데이터를 위한 유저 데이터 얻기
- Markdown을 적고 코드를 정리하는 과정에서 약간의 문제가 있을 수도 있음. (확인하는데 시간이 오래걸려서 생략)
- file을 저장하는 과정에서 utf8로 저장한 경우와 아닌 경우가 섞여있어서 파일이 동시에 열리지 않는 경우가 생김
--> utf8로 저장할 경우 엑셀로 실행하거나 encoding=utf8 로 열지 않으면 깨져서 보임.(메모장 또는 read_csv를 할 때 encoding=utf8을 붙여주자)
## 1. 유저 닉네임 얻기
- 랭킹 정보를 이용하여 6~1000위까지의 유저의 정보를 얻음
- 1~5위는 제공되는 html 형태가 달라서 큰 영향이 없을거라고 생각하고 생략함.
- http://www.op.gg/ranking/ladder/

In [ ]:
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np
import glob

In [2]:
# get user list
user_list = [] # 순위권에 들어가는 유저 닉네임
for i in range(1,11):
    url = 'http://www.op.gg/ranking/ladder/page={0}'.format(i)
    resp = requests.get(url)
    bs = BeautifulSoup(resp.text, 'html.parser')
    users = bs.findAll("td", {"class": "ranking-table__cell ranking-table__cell--summoner"})
    for line in users:
        line = str(line)
        line = line[line.find("<span>"):line.find("</span>")].replace("<span>","")
        user_list.append(line)
user_list[:5]

['deft123456', 'Tar2an', 'SKT T1 Untara', '11 14 9 7 8 20', 'DWG BeryL']

## 2. 유저 닉네임을 기준으로 경기데이터(match)를 얻을 유저를 무작위로 선정
- 총 995명(6~1000) 중에서 900명을 대상으로 무작위로 사람을 뽑음.
- 현재 마지막 발표에서는 무작위로 10명을 뽑아서 최근 2경기를 뽑아서 20경기를 수집 + 이전에 수동으로 뽑은 6경기를 더함.

### 1) 무작위로 사람 뽑기

In [3]:
u_name = [] # 무작위로 뽑은 사람 닉네임
for i in range(10):
    u_name.append(user_list[int(np.random.uniform(900))])
len(u_name)

10

In [4]:
u_name

['집밖에나갈일없음',
 'oahizdasfaswzxeo',
 '길동이에유',
 'Boldness',
 'SongGoon',
 'FJHGGJ',
 '견지해 나가다',
 '오함마드알리',
 'mcmglllc',
 'BtC RYAN']

### 2) 뽑은 사람의 최근 전적에 들어가는 다른 사람들과 경기 결과(승,패)를 수집 
- 사용하려면 selenium을 사용하기 위한 driver를 설치하고 경로를 지정해줘야함.

In [5]:
driver_address = 'D:\chromedriver' ## 설치후 경로설정 필요
match = []
match_member = []
match_result = []
trait = 0
result_idx = 0
for i in u_name:
    url = 'http://www.op.gg/summoner/userName={0}'.format(i)
    driver = webdriver.Chrome(driver_address)## 주소바꿔주기
    driver.implicitly_wait(3)
    driver.get(url)
    driver.find_element_by_xpath('//a[@data-type="soloranked"]').click()
    time.sleep(1.5)
    html = driver.page_source
    bs = BeautifulSoup(html, 'html.parser')
    match_users = bs.findAll("div", {"class": "SummonerName"})
    result = bs.findAll("div", {"class": "GameResult"})
    for line in result:
        result_idx += 1
        r = line.text.strip()
        match_result.append(r)
        if result_idx % 2 == 0:
            break
    tmp = []
    for line in match_users:
        trait += 1
        u = line.text.strip()
        tmp.append(u)
        match.append(u)
        if trait % 10 == 0:
            match_member.append(tmp)
            tmp = []
        if trait % 20 == 0:
            break
    time.sleep(1)
    driver.quit()
    
len(match)

200

In [6]:
print(len(match))
print(len(match_member))
print(len(match_result))

200
20
20


In [30]:
match[:20]

['집밖에나갈일없음',
 'Sim Student',
 'KDM PERT',
 'ARE2A',
 '라인전이기니갱와',
 '이재훈다딱',
 'jinwandalaohu',
 'whisper snow',
 'JAG Wraith',
 'supbest',
 '집밖에나갈일없음',
 'Ningxia Allah',
 'mcmglllc',
 '수지Suzy',
 'ARE2A',
 '이재훈다딱',
 'jinwandalaohu',
 'KDM PERT',
 '시공 여행자',
 'supbest']

## 3. 앞에서 얻은 경기에 참여한 유저의 최근 전적을 수집
- 개인당 20경기(1페이지)를 수집하며, 사람에 따라 적을수도 있음.
- http://www.op.gg/summoner/userName=

### 1) 전적 정보를 긁어오기 위한 설정
- driver_address : 앞과 마찬가지로 selenium 사용에 필요한 드라이버 설치후 경로 설정
- start_number : 넣어준 리스트의 시작 지점
- end_number : 넣어준 리스트의 종료 지점
- match_number_start : 경기를 구분시켜줄 match_id

- 크롤링 도중에 전적 사이트 서버 오류나 인터넷 반응 오류 등등 여러가지 오류가 발생
- 1명의 정보를 얻을 때마다 완료지점을 print하여 거기부터 다시 이어서 함.

In [13]:
driver_address = 'D:\chromedriver'
## 수정할것 3가지 (오류 발생시 or 원하는 사람만 긁을 때)
start_number = 1
end_number = len(match)
match_number_start = 362128

- 데이터가 저장될 경로 설정하기 (file_name)

In [9]:
def match_data_load(user_name, match_number_start):
    file_name = "C:\\Users\\waf\\Documents\\00_DGIST 수업\\03_02\\Big data\\프로젝트\\2017_BigData\\Data\\match\\Mat_Set\\user_data_{0}.csv".format(user_name)
    
    driver = webdriver.Chrome(driver_address)## 주소바꿔주기
    driver.implicitly_wait(3)
    driver.get('http://www.op.gg/summoner/userName={0}'.format(user_name))
    
    driver.find_element_by_xpath('//a[@data-type="soloranked"]').click()
    
    game_detail_button = driver.find_elements_by_css_selector('a.Button.MatchDetail')
    for a in game_detail_button:
        a.click()
        time.sleep(1)
    time.sleep(3)    
    html = driver.page_source
    PlayerData = BeautifulSoup(html, 'html.parser')
#     print(PlayerData.prettify())
    team_raw = PlayerData.find_all('tbody', {'class':'Content'})
    game_raw = PlayerData.find_all('thead', {'class':'Header'})
    object_raw = PlayerData.find_all('div', {'class':'Summary'})

    win_lose = []
    champion_list = []
    name_list = []
    kill_list = []
    death_list = []
    assist_list = []
    ###
    damage_list = []
    buy_ward_list = []
    build_ward_list1 = []
    build_ward_list2 = []
    CS_list = []
    CSperMin_list = []
    object_list1 = []
    object_list2 = []
    object_list3 = []

    retry = 0
    index = []
    idx = 0
    for gline in game_raw:
        idx += 1
        game_result_raw = gline.find_all('span', {'class':'GameResult'})

        game_result = []
        for gresult in game_result_raw:
            gresult = gresult.text.strip()
            if (gresult != "승리") & (gresult != "패배"):
                retry += 1
                index.append(idx)
                continue
            game_result.append(gresult)
        if retry == 0:
            win_lose.append(game_result)
        else:
            retry -= 1

    for obj in object_raw:
        idx = 0
        for i in obj.find_all('div',{'class':'ObjectScore'}):
            tmp = []
            if idx == 0:
                idx += 1
                for j in range(5):
                    tmp.append(i.text.strip())
                object_list1.append(tmp)
            elif idx == 1:
                idx += 1
                for j in range(5):
                    tmp.append(i.text.strip())
                object_list2.append(tmp)
            elif idx == 2:
                idx = 0
                for j in range(5):
                    tmp.append(i.text.strip())
                object_list3.append(tmp)    
    idx=0
    for line in team_raw:
        idx += 1
        if idx in index:
            continue
        teamdata = []
        team_champion_raw = line.find_all('td', {'class':'ChampionImage Cell'})
        team_name_raw = line.find_all('td', {'class':'SummonerName Cell'})
        team_kill_raw = line.find_all('span',{'class':'Kill'})
        team_death_raw = line.find_all('span',{'class':'Death'})
        team_assist_raw = line.find_all('span',{'class':'Assist'})
        ### 추가한것
        team_damage_raw = line.find_all('div',{'class':'ChampionDamage'})
        team_buy_ward_raw = line.find_all('div',{'class':'Buy'})
        team_build_ward_raw = line.find_all('div',{'class':'Stats'})
        team_CS_raw = line.find_all('div',{'class':'CS'})
        team_CSperMin_raw = line.find_all('div',{'class':'CSPerMinute'})

        team_champion = []
        for champ in team_champion_raw:
            champ = champ.text.strip()
            champ = champ[:champ.find('\n')]
            team_champion.append(champ)
        champion_list.append(team_champion)

        team_name = []
        for name in team_name_raw:
            name = name.text.strip()
            team_name.append(name)
        name_list.append(team_name)

        team_kill_participation = []

        team_kill = []
        for kills in team_kill_raw:
            kills = kills.text.strip()
            team_kill.append(kills)
        kill_list.append(team_kill)

        team_death = []
        for deaths in team_death_raw:
            deaths = deaths.text.strip()
            team_death.append(deaths)
        death_list.append(team_death)

        team_assist = []
        for assists in team_assist_raw:
            assists = assists.text.strip()
            team_assist.append(assists)
        assist_list.append(team_assist)

        team_damage = []
        for damage in team_damage_raw:
            damage = damage.text.strip().replace(",","")
            team_damage.append(damage)
        damage_list.append(team_damage)   

        team_buy_ward = []
        for buy_ward in team_buy_ward_raw:
            buy_ward = buy_ward.text.strip()
            team_buy_ward.append(buy_ward)
        buy_ward_list.append(team_buy_ward)  

        team_build_ward1 = []
        team_build_ward2 = []
        for build_ward in team_build_ward_raw:
            build_ward = build_ward.text.strip().replace("\t","").replace("\n","").replace("/","")
            team_build_ward1.append(build_ward[:build_ward.find(" ")])
            team_build_ward2.append(build_ward[build_ward.find(" "):].replace(" ",""))
        build_ward_list1.append(team_build_ward1)
        build_ward_list2.append(team_build_ward2)

        team_CS = []
        for CS in team_CS_raw:
            CS = CS.text.strip()
            team_CS.append(CS)
        CS_list.append(team_CS)  

        team_CSperMin = []
        for CSperMin in team_CSperMin_raw:
            CSperMin = CSperMin.text.strip()
            team_CSperMin.append(CSperMin[build_ward.find(" "):].replace(" ","").replace("당",""))
        CSperMin_list.append(team_CSperMin)  

    win_lose = win_lose[1:]

    KDAsorted = []
    matchorder = match_number_start
    matchnumber = 1
    for order in range(int((len(win_lose)+1)/2)):
        matchnumber = matchnumber-1
        for i in range(5):
            team_member = []
            team_member.append(matchorder)
            team_member.append(win_lose[matchnumber][0])
            team_member.append(champion_list[matchnumber][i])
            team_member.append(name_list[matchnumber][i])
            team_member.append(int(kill_list[matchnumber][i]))
            team_member.append(int(death_list[matchnumber][i]))
            team_member.append(int(assist_list[matchnumber][i]))
            team_member.append(int(damage_list[matchnumber][i]))
            team_member.append(int(buy_ward_list[matchnumber][i]))
            team_member.append(int(build_ward_list1[matchnumber][i]))
            team_member.append(int(build_ward_list2[matchnumber][i]))
            team_member.append(int(CS_list[matchnumber][i]))
            team_member.append(float(0))
            team_member.append(int(object_list1[matchnumber][i]))
            team_member.append(int(object_list2[matchnumber][i]))
            team_member.append(int(object_list3[matchnumber][i]))
            KDAsorted.append(team_member)
        matchnumber += 1
        for i in range(5):
            team_member = []
            team_member.append(matchorder)
            team_member.append(win_lose[matchnumber][0])
            team_member.append(champion_list[matchnumber][i])
            team_member.append(name_list[matchnumber][i])
            team_member.append(int(kill_list[matchnumber][i]))
            team_member.append(int(death_list[matchnumber][i]))
            team_member.append(int(assist_list[matchnumber][i]))
            team_member.append(int(damage_list[matchnumber][i]))
            team_member.append(int(buy_ward_list[matchnumber][i]))
            team_member.append(int(build_ward_list1[matchnumber][i]))
            team_member.append(int(build_ward_list2[matchnumber][i]))
            team_member.append(int(CS_list[matchnumber][i]))
            team_member.append(float(0))
            team_member.append(int(object_list1[matchnumber][i]))
            team_member.append(int(object_list2[matchnumber][i]))
            team_member.append(int(object_list3[matchnumber][i]))
            KDAsorted.append(team_member)
        matchnumber += 2
        matchorder += 1

    kda_data = pd.DataFrame(KDAsorted, columns=['MatchNumber','GameResult', 'Champion', 'Nickname', 
                                                'Kill', 'Death', 'Assist', 'Damage','Buy_Pward','Build_ward','Destroy_ward',
                                                'CS','CSperMin','Baron','Dragon','Tower'])
    # kda_data['KDA'] = (kda_data['Kill'] + kda_data['Assist'])/ kda_data['Death']
    # kda_data.head(10)
    
    kda_data.to_csv(file_name)
    driver.quit()
    return matchorder

- 선수를 정보 얻기
- 선수 1명을 완료할 때마다 1) 완료 지점, 에러가 날 경우 시작할 지점의 2) count와 3) start_match_number를 알려줌.

In [14]:
count = 1; 
for user_name in match:
    if count == end_number+1:
        break
    if count < start_number:
        count += 1
        continue
    match_number_start = match_data_load(user_name, match_number_start)
    print("complete! count :",count,", error_start : count",count+1,", start_number", match_number_start)
    count += 1

complete! count : 108 , error_start : count 109 , start_number 362148
complete! count : 109 , error_start : count 110 , start_number 362168
complete! count : 110 , error_start : count 111 , start_number 362188
complete! count : 111 , error_start : count 112 , start_number 362208
complete! count : 112 , error_start : count 113 , start_number 362228
complete! count : 113 , error_start : count 114 , start_number 362248
complete! count : 114 , error_start : count 115 , start_number 362268
complete! count : 115 , error_start : count 116 , start_number 362288
complete! count : 116 , error_start : count 117 , start_number 362308
complete! count : 117 , error_start : count 118 , start_number 362328
complete! count : 118 , error_start : count 119 , start_number 362347
complete! count : 119 , error_start : count 120 , start_number 362367
complete! count : 120 , error_start : count 121 , start_number 362387
complete! count : 121 , error_start : count 122 , start_number 362407
complete! count : 12

## 4. match data 만들기
- 앞에서 얻은 데이터를 불러옴.
- 선수 이름 닉네임을 기준으로 그룹을 묶은 후 각 지표의 평균값을 계산하여 저장함.

### 1) 데이터 불러오기

In [33]:
# get data file names
path =r'C:\Users\waf\Documents\00_DGIST 수업\03_02\Big data\프로젝트\2017_BigData\Data\match\Mat_Set'
filenames = glob.glob(path + "/*.csv")
filenames
dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename, engine='python'))
# Concatenate all data into one DataFrame
kda_data = pd.concat(dfs, ignore_index=True)
kda_data.head()

,Unnamed: 0,MatchNumber,GameResult,Champion,Nickname,Kill,Death,Assist,Damage,Buy_Pward,Build_ward,Destroy_ward,CS,CSperMin,Baron,Dragon,Tower
0,0,361635,패배,잭스,1 Peak,5,9,1,10256,1,6,2,150,0.0,0,0,0
1,1,361635,패배,세주아니,어린여자가좋아,1,4,2,5158,2,37,4,96,0.0,0,0,0
2,2,361635,패배,아리,Haide yapI,1,5,2,9240,1,6,1,185,0.0,0,0,0
3,3,361635,패배,자야,Act Different,0,5,4,6148,1,6,1,152,0.0,0,0,0
4,4,361635,패배,알리스타,LUALU,2,4,3,2694,5,35,3,33,0.0,0,0,0


### 2) 선수별로 각 지표의 평균을 구함
- 추가적으로 KDA, KDA2(K+0.2A-1.2D) 지표도 추가

In [35]:
match_data = kda_data.groupby('Nickname')[['Kill','Death','Assist','Damage','Buy_Pward','Build_ward','Destroy_ward','CS','Baron','Dragon','Tower']].mean()
match_data['Death_fake'] = match_data['Death']
match_data['Death_fake'] = match_data['Death_fake'].replace(0, 1)
match_data['KDA'] = (match_data['Kill'] + match_data['Assist'])/ match_data['Death_fake']
match_data['KDA2'] = match_data['Kill'] + 0.2*match_data['Assist'] - 1.2*match_data['Death']
match_data = match_data.drop('Death_fake', 'columns')
match_data.head()

,Kill,Death,Assist,Damage,Buy_Pward,Build_ward,Destroy_ward,CS,Baron,Dragon,Tower,KDA,KDA2
Nickname,,,,,,,,,,,,,
0 Day,3.00,4.00,5.000,13320.0,1.000,9.000,4.000,227.000,0.000,0.000,2.000,2.000000,-0.800
000811,4.00,3.00,7.000,11194.0,3.000,33.000,3.000,183.000,2.000,1.000,10.000,3.666667,1.800
0077777777,11.25,5.25,9.125,18361.0,1.375,7.375,3.875,142.125,0.375,1.625,4.375,3.880952,6.775
03년생 패드리퍼,3.00,4.00,2.000,7060.0,0.000,5.500,2.500,90.000,0.000,0.000,4.000,1.250000,-1.400
0722,6.00,7.00,16.000,26590.0,4.000,9.500,3.000,158.500,1.000,1.000,10.000,3.142857,0.800


### 3) 앞에서 얻었던 경기별로 유저수 정보를 이용하여 match 파일을 만듬
- 앞에서 크롤링할때와 같은 순서로 선수들 배열

In [37]:
match_number = 19
for i,m in enumerate(match_member):
    match_number += 1
    match1 = match_data.loc[m]
    match1.to_csv('C:\\Users\\waf\\Documents\\00_DGIST 수업\\03_02\\Big data\\프로젝트\\2017_BigData\\Data\\match\\match\\match{0}.csv'.format(match_number))
match1

,Kill,Death,Assist,Damage,Buy_Pward,Build_ward,Destroy_ward,CS,Baron,Dragon,Tower,KDA,KDA2
Nickname,,,,,,,,,,,,,
Shi Yue,4.410959,4.479452,4.863014,17562.383562,0.739726,8.164384,2.397260,184.547945,0.246575,0.917808,3.534247,2.070336,0.008219
내 마음 아프니까,4.205128,6.615385,6.871795,15242.153846,0.807692,15.448718,4.961538,163.179487,0.346154,0.743590,3.833333,1.674419,-2.358974
TeachersWu,3.742857,5.157143,5.914286,12289.857143,3.800000,16.528571,5.514286,158.414286,0.314286,1.242857,3.200000,1.872576,-1.262857
혼자가 편해졌어,3.769231,5.875000,6.423077,14975.192308,3.721154,18.634615,7.105769,172.086538,0.528846,0.625000,3.759615,1.734861,-1.996154
YOYOuuuuuO,2.211765,4.388235,8.082353,7544.000000,3.647059,16.576471,4.447059,74.929412,0.164706,0.729412,3.447059,2.345845,-1.437647
Alielie,4.475000,6.112500,4.650000,13866.562500,2.500000,9.412500,2.512500,175.037500,0.550000,1.012500,4.875000,1.492843,-1.930000
BtC RYAN,4.765432,4.407407,9.037037,11695.654321,4.370370,25.432099,7.432099,139.753086,0.456790,0.888889,5.382716,3.131653,1.283951
BtC Hipo,4.425287,3.160920,7.862069,15344.505747,4.091954,15.873563,4.321839,191.402299,0.724138,0.942529,6.413793,3.887273,2.204598
너만재미써,8.477612,3.014925,8.164179,23603.940299,2.194030,8.223881,5.238806,229.104478,0.671642,1.328358,7.134328,5.519802,6.492537


### 4) Test 확인을 위한 승패 결과 얻기
- 실제로 사용할 때는 list를 복사해서 사용함.

In [39]:
match_result
result = []
for i in match_result:
    if i == "승리":
        result.append(1)
    else:
        result.append(0)
result

[1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1]

In [40]:
m_result = pd.DataFrame()
m_result["result"] = result
m_result.to_csv('C:\\Users\\waf\\Documents\\00_DGIST 수업\\03_02\\Big data\\프로젝트\\2017_BigData\\Data\\match\\match\\match{0}_result.csv'.format(match_number))